# **Import Libraries**

In [24]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import time

In [26]:
np.random.seed(42)

In [28]:
import tensorflow as tf

tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

# **Data Pre-processing**

## Import Dataset

In [30]:
# import the DataSet
dt = pd.read_csv('Interpolated_Training_Data.csv')

In [32]:
dt = dt.dropna()
dt

,Configuration,Pressure,Theta1,Theta2,Theta3,Theta4,Theta5,Theta6
0,DS10DS10_0_1,0.0,-14.266649,-8.307888,-5.844170,-4.538399,-3.596456,-3.205699
1,DS10DS10_0_1,8.5,-14.408452,-8.399550,-5.738773,-4.142236,-2.953373,-2.136300
2,DS10DS10_0_1,17.0,-13.957240,-7.325239,-4.092228,-2.035293,-0.421883,0.750848
3,DS10DS10_0_1,25.5,-13.576226,-6.441465,-2.790626,-0.286894,1.695869,3.151817
4,DS10DS10_0_1,34.0,-13.579100,-6.399944,-2.623292,0.035310,2.201195,3.861465
...,...,...,...,...,...,...,...,...
4177,E30E30_3_6,306.0,-15.068790,-9.740283,-6.932789,-5.844170,-5.786874,-6.119189
4178,E30E30_3_6,314.5,-15.068790,-9.740283,-6.932789,-5.892871,-5.844170,-6.179350
4179,E30E30_3_6,323.0,-15.126086,-9.740283,-6.932789,-5.901465,-5.844170,-6.296806
4180,E30E30_3_6,331.5,-15.068790,-9.740283,-6.932789,-5.844170,-5.789739,-6.356967


In [34]:
dt[dt.columns[7]].to_numpy()

array([-3.20569886, -2.13630041,  0.75084829, ..., -6.29680617,
       -6.35696674, -6.18794419])

## Divide Dataset into Data and Target Sets

In [35]:
# Dividing Data and Target set
data_0 = dt[['Pressure', 'Theta6']].to_numpy()
data_1 = dt[['Pressure', 'Theta1', 'Theta6']].to_numpy()
data_2 = dt[['Pressure', 'Theta1', 'Theta2', 'Theta6']].to_numpy()
data_3 = dt[['Pressure', 'Theta1', 'Theta2', 'Theta3', 'Theta6']].to_numpy()
data_4 = dt[['Pressure', 'Theta1', 'Theta2', 'Theta3', 'Theta4', 'Theta6']].to_numpy()
data_5 = dt[['Pressure', 'Theta1', 'Theta2', 'Theta3', 'Theta4', 'Theta5', 'Theta6']].to_numpy()
target = dt['Configuration'].to_numpy()
dt['Configuration'].unique()

array(['DS10DS10_0_1', 'DS10DS10_0_2', 'DS10DS10_0_3', 'DS10DS10_0_6',
       'DS10DS10_1_2', 'DS10DS10_1_3', 'DS10DS10_1_6', 'DS10DS10_2_3',
       'DS10DS10_2_6', 'DS10DS10_3_6', 'DS10DS30_1_1', 'DS10DS30_1_2',
       'DS10DS30_1_3', 'DS10DS30_1_6', 'DS10DS30_2_1', 'DS10DS30_2_2',
       'DS10DS30_2_3', 'DS10DS30_2_6', 'DS10DS30_3_1', 'DS10DS30_3_2',
       'DS10DS30_3_3', 'DS10DS30_3_6', 'DS10DS30_6_1', 'DS10DS30_6_2',
       'DS10DS30_6_3', 'DS10DS30_6_6', 'DS10E30_1_1', 'DS10E30_1_2',
       'DS10E30_1_3', 'DS10E30_2_1', 'DS10E30_2_2', 'DS10E30_2_3',
       'DS10E30_2_6', 'DS10E30_3_1', 'DS10E30_3_2', 'DS10E30_3_3',
       'DS10E30_3_6', 'DS10E30_6_1', 'DS10E30_6_3', 'DS10E30_6_6',
       'DS20DS20_0_1', 'DS20DS20_0_2', 'DS20DS20_0_3', 'DS20DS20_0_6',
       'DS20DS20_1_2', 'DS20DS20_1_3', 'DS20DS20_1_6', 'DS20DS20_2_3',
       'DS20DS20_2_6', 'DS20DS20_3_6', 'DS20E30_1_1', 'DS20E30_1_2',
       'DS20E30_1_3', 'DS20E30_1_6', 'DS20E30_2_1', 'DS20E30_2_2',
       'DS20E30_2_6', 'DS2

In [36]:
np.shape(dt['Configuration'].unique())

(102,)

## Decoding

In [37]:
from sklearn import preprocessing
from keras.utils import to_categorical

le = preprocessing.LabelEncoder()

categories = dt['Configuration'].unique()

# Fit the encoder to the categories and transform (encode) them
encoded_labels = le.fit_transform(categories)

# Now to decode, use inverse_transform
decoded_labels = le.inverse_transform(encoded_labels)

# Print encoded and decoded results for comparison
print("Encoded Labels:", encoded_labels)
print("Decoded Labels:", decoded_labels)

Encoded Labels: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101]
Decoded Labels: ['DS10DS10_0_1' 'DS10DS10_0_2' 'DS10DS10_0_3' 'DS10DS10_0_6'
 'DS10DS10_1_2' 'DS10DS10_1_3' 'DS10DS10_1_6' 'DS10DS10_2_3'
 'DS10DS10_2_6' 'DS10DS10_3_6' 'DS10DS30_1_1' 'DS10DS30_1_2'
 'DS10DS30_1_3' 'DS10DS30_1_6' 'DS10DS30_2_1' 'DS10DS30_2_2'
 'DS10DS30_2_3' 'DS10DS30_2_6' 'DS10DS30_3_1' 'DS10DS30_3_2'
 'DS10DS30_3_3' 'DS10DS30_3_6' 'DS10DS30_6_1' 'DS10DS30_6_2'
 'DS10DS30_6_3' 'DS10DS30_6_6' 'DS10E30_1_1' 'DS10E30_1_2' 'DS10E30_1_3'
 'DS10E30_2_1' 'DS10E30_2_2' 'DS10E30_2_3' 'DS10E30_2_6' 'DS10E30_3_1'
 'DS10E30_3_2' 'DS10E30_3_3' 'DS10E30_3_6' 

## Label Encoding

In [38]:
# Label encoding

le.fit(target)
target_encoded = to_categorical(le.transform(target), 102) # Number has to be changed
np.shape(target_encoded)

(4182, 102)

In [39]:
target_encoded

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

## Split Train & Validation Set

In [40]:
# Shuffle the data and target
permutation = tf.random.shuffle(tf.range(tf.shape(data_0)[0]))

# Shuffle Data
data_shuffled_0 = tf.gather(data_0, permutation)
data_shuffled_1 = tf.gather(data_1, permutation)
data_shuffled_2 = tf.gather(data_2, permutation)
data_shuffled_3 = tf.gather(data_3, permutation)
data_shuffled_4 = tf.gather(data_4, permutation)
data_shuffled_5 = tf.gather(data_5, permutation)

# Shuffle Target
target_shuffled = tf.gather(target_encoded, permutation)

print(data_shuffled_0)
print(target_shuffled)

# Split train and Validation Set
train_data_0 = data_shuffled_0
train_data_1 = data_shuffled_1
train_data_2 = data_shuffled_2
train_data_3 = data_shuffled_3
train_data_4 = data_shuffled_4
train_data_5 = data_shuffled_5

train_target = target_shuffled

tf.Tensor(
[[ 68.         -11.43623759]
 [170.           8.3938317 ]
 [212.5         -9.45380362]
 ...
 [340.         -44.175046  ]
 [ 17.          -3.45043779]
 [204.         -27.22695442]], shape=(4182, 2), dtype=float64)
tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(4182, 102), dtype=float64)


In [41]:
# Calculate the number of samples for validation set (20% of the data)\
num_val_samples = int(0.2 * int(tf.shape(data_0)[0]))

# Get the validation data and target
val_data_0 = data_shuffled_0[:num_val_samples]
val_data_1 = data_shuffled_1[:num_val_samples]
val_data_2 = data_shuffled_2[:num_val_samples]
val_data_3 = data_shuffled_3[:num_val_samples]
val_data_4 = data_shuffled_4[:num_val_samples]
val_data_5 = data_shuffled_5[:num_val_samples]

val_target = target_shuffled[:num_val_samples]

## Standardlization

In [42]:
# Standarlization

from sklearn.preprocessing import StandardScaler
def Standarlization(data, train_data, val_data):
    ss = StandardScaler()
    ss.fit(data)
    train_scaled = ss.transform(train_data)
    val_scaled = ss.transform(val_data)
    return train_scaled, val_scaled

train_scaled_0, val_scaled_0 = Standarlization(data_0, train_data_0, val_data_0)
train_scaled_1, val_scaled_1 = Standarlization(data_1, train_data_1, val_data_1)
train_scaled_2, val_scaled_2 = Standarlization(data_2, train_data_2, val_data_2)
train_scaled_3, val_scaled_3 = Standarlization(data_3, train_data_3, val_data_3)
train_scaled_4, val_scaled_4 = Standarlization(data_4, train_data_4, val_data_4)
train_scaled_5, val_scaled_5 = Standarlization(data_5, train_data_5, val_data_5)

In [43]:
print(train_scaled_0.shape, train_target.shape)

(4182, 2) (4182, 102)


# **Create FNN Model**

In [44]:
#checkpoint_cb = keras.callbacks.ModelCheckpoint('/content/my_model.keras')
#early_stopping_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

In [57]:
# Model
def Model (train_scaled, train_target, val_scaled, val_target, input_dim):
  # Define the model
  model = Sequential()
  model.add(Dense(512, input_dim=input_dim, activation=keras.layers.LeakyReLU(alpha=0.01))) # Number need to be changed depend on training data columns
  model.add(Dropout(0.2))
  model.add(Dense(100, activation=keras.layers.LeakyReLU(alpha=0.01)))
  model.add(Dense(102, activation='softmax')) # Number has to be changed depends on .unique()

  # compile model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  # Record the start time
  start_time = time.time()

  # Model Fit
  history = model.fit(train_scaled, train_target, epochs=50,
                    validation_data=(val_scaled, val_target)) #,
                    #callbacks=[checkpoint_cb, early_stopping_cb])

  # Record the end time
  end_time = time.time()

  # Calculate the duration
  duration = end_time - start_time

  # Evaluate the model
  loss, accuracy = model.evaluate(val_scaled, val_target)

  return model, history, loss, accuracy, duration

In [58]:
# Define the model
model = Sequential()
model.add(Dense(512, input_dim=7, activation=keras.layers.LeakyReLU(alpha=0.01))) # Number need to be changed depend on training data columns
model.add(Dropout(0.2))
model.add(Dense(100, activation=keras.layers.LeakyReLU(alpha=0.01)))
model.add(Dense(102, activation='softmax')) # Number has to be changed depends on .unique()


In [59]:
# compile model
from keras.optimizers import SGD

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## Model fit with 0 Additional Parameter

In [60]:
# Model Fit
model_0, history_0, loss_0, accuracy_0, duration_0 = Model(train_scaled_0, train_target, val_scaled_0, val_target, 2)

Epoch 1/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.0161 - loss: 4.5030 - val_accuracy: 0.0275 - val_loss: 3.9768
Epoch 2/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.0296 - loss: 3.9096 - val_accuracy: 0.0682 - val_loss: 3.5398
Epoch 3/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.0553 - loss: 3.5381 - val_accuracy: 0.1148 - val_loss: 3.2733
Epoch 4/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.0706 - loss: 3.3193 - val_accuracy: 0.1220 - val_loss: 3.1046
Epoch 5/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0755 - loss: 3.1813 - val_accuracy: 0.1316 - val_loss: 2.9865
Epoch 6/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0909 - loss: 3.0785 - val_accuracy: 0.1411 - val_loss: 2.8925
Epoch 7/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0997 - loss: 2.9996 - val_accuracy: 0.1483 - val_loss: 2.8166
Epoch 8/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1049 - loss: 2.9364 - val_accuracy: 0.

## Model fit With 1 Additional Parameter

In [61]:
model_1, history_1, loss_1, accuracy_1, duration_1 = Model(train_scaled_1, train_target, val_scaled_1, val_target, 3)

Epoch 1/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.0222 - loss: 4.4318 - val_accuracy: 0.0478 - val_loss: 3.6850
Epoch 2/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0787 - loss: 3.4833 - val_accuracy: 0.1902 - val_loss: 2.7801
Epoch 3/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1912 - loss: 2.7332 - val_accuracy: 0.3218 - val_loss: 2.2955
Epoch 4/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2581 - loss: 2.3613 - val_accuracy: 0.3852 - val_loss: 2.0246
Epoch 5/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3236 - loss: 2.1363 - val_accuracy: 0.4414 - val_loss: 1.8338
Epoch 6/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3578 - loss: 1.9708 - val_accuracy: 0.4797 - val_loss: 1.6930
Epoch 7/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3896 - loss: 1.8406 - val_accuracy: 0.4844 - val_loss: 1.5934
Epoch 8/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.4097 - loss: 1.7471 - val_accuracy: 0.

## Model fit With 2 Additional Parameter

In [62]:
model_2, history_2, loss_2, accuracy_2, duration_2 = Model(train_scaled_2, train_target, val_scaled_2, val_target, 4)

Epoch 1/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.0198 - loss: 4.4223 - val_accuracy: 0.0574 - val_loss: 3.6977
Epoch 2/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0863 - loss: 3.5000 - val_accuracy: 0.2440 - val_loss: 2.7295
Epoch 3/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2208 - loss: 2.6829 - val_accuracy: 0.3541 - val_loss: 2.2177
Epoch 4/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3055 - loss: 2.2661 - val_accuracy: 0.4378 - val_loss: 1.9210
Epoch 5/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3795 - loss: 2.0107 - val_accuracy: 0.5096 - val_loss: 1.7011
Epoch 6/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4241 - loss: 1.8243 - val_accuracy: 0.5682 - val_loss: 1.5438
Epoch 7/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4687 - loss: 1.6696 - val_accuracy: 0.6077 - val_loss: 1.3935
Epoch 8/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4944 - loss: 1.5489 - val_accuracy: 0.

## Model fit With 3 Additional Parameter

In [63]:
model_3, history_3, loss_3, accuracy_3, duration_3 = Model(train_scaled_3, train_target, val_scaled_3, val_target, 5)

Epoch 1/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.0233 - loss: 4.3855 - val_accuracy: 0.0718 - val_loss: 3.7173
Epoch 2/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0908 - loss: 3.5213 - val_accuracy: 0.2201 - val_loss: 2.7380
Epoch 3/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2098 - loss: 2.6794 - val_accuracy: 0.3266 - val_loss: 2.2293
Epoch 4/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2892 - loss: 2.2724 - val_accuracy: 0.4007 - val_loss: 1.9446
Epoch 5/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3524 - loss: 2.0273 - val_accuracy: 0.4809 - val_loss: 1.7180
Epoch 6/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4034 - loss: 1.8482 - val_accuracy: 0.5287 - val_loss: 1.5712
Epoch 7/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4358 - loss: 1.7114 - val_accuracy: 0.5538 - val_loss: 1.4312
Epoch 8/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4846 - loss: 1.5751 - val_accuracy: 0.

## Model fit With 4 Additional Parameter

In [64]:
model_4, history_4, loss_4, accuracy_4, duration_4 = Model(train_scaled_4, train_target, val_scaled_4, val_target, 6)

Epoch 1/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.0265 - loss: 4.3990 - val_accuracy: 0.0825 - val_loss: 3.5728
Epoch 2/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1003 - loss: 3.3500 - val_accuracy: 0.2727 - val_loss: 2.6023
Epoch 3/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2190 - loss: 2.5919 - val_accuracy: 0.3876 - val_loss: 2.1300
Epoch 4/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.3185 - loss: 2.2166 - val_accuracy: 0.4426 - val_loss: 1.8602
Epoch 5/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.3800 - loss: 1.9788 - val_accuracy: 0.4988 - val_loss: 1.6504
Epoch 6/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4354 - loss: 1.7788 - val_accuracy: 0.5754 - val_loss: 1.4641
Epoch 7/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4704 - loss: 1.6379 - val_accuracy: 0.6268 - val_loss: 1.3288
Epoch 8/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5214 - loss: 1.5060 - val_accuracy: 0.

## Model fit With 5 Additional Parameter

In [65]:
model_5, history_5, loss_5, accuracy_5, duration_5 = Model(train_scaled_5, train_target, val_scaled_5, val_target, 7)

Epoch 1/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.0218 - loss: 4.3944 - val_accuracy: 0.0885 - val_loss: 3.6285
Epoch 2/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1047 - loss: 3.3980 - val_accuracy: 0.2488 - val_loss: 2.6320
Epoch 3/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2197 - loss: 2.5998 - val_accuracy: 0.3577 - val_loss: 2.1436
Epoch 4/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.3082 - loss: 2.2082 - val_accuracy: 0.4510 - val_loss: 1.8493
Epoch 5/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.3769 - loss: 1.9652 - val_accuracy: 0.5203 - val_loss: 1.6400
Epoch 6/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.4423 - loss: 1.7829 - val_accuracy: 0.5897 - val_loss: 1.4525
Epoch 7/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.4867 - loss: 1.6285 - val_accuracy: 0.6304 - val_loss: 1.3153
Epoch 8/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5204 - loss: 1.4911 - val_accuracy: 0.

# **Compare Accuracies**

In [66]:
result_model0 = model_0.evaluate(val_scaled_0, val_target)
result_model1 = model_1.evaluate(val_scaled_1, val_target)
result_model2 = model_2.evaluate(val_scaled_2, val_target)
result_model3 = model_3.evaluate(val_scaled_3, val_target)
result_model4 = model_4.evaluate(val_scaled_4, val_target)
result_model5 = model_5.evaluate(val_scaled_5, val_target)

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3064 - loss: 1.9668 
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8003 - loss: 0.6358 
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8923 - loss: 0.3397 
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8866 - loss: 0.3559 
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9157 - loss: 0.3113 
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9167 - loss: 0.3094 


In [71]:
print("Add 0 parameter: ")
print("Loss: ", result_model0[0])
print("Accuracy: ", np.round(result_model0[1]*100, 2), "%")

print("Add 1 parameter: ")
print("Loss: ", result_model1[0])
print("Accuracy: ", np.round(result_model1[1]*100, 2), "%")

print("Add 2 parameter: ")
print("Loss: ", result_model2[0])
print("Accuracy: ", np.round(result_model2[1]*100, 2), "%")

print("Add 3 parameter: ")
print("Loss: ", result_model3[0])
print("Accuracy: ", np.round(result_model3[1]*100, 2), "%")

print("Add 4 parameter: ")
print("Loss: ", result_model4[0])
print("Accuracy: ", np.round(result_model4[1]*100, 2), "%")

print("Add 5 parameter: ")
print("Loss: ", result_model5[0])
print("Accuracy: ", np.round(result_model5[1]*100, 2), "%")

Add 0 parameter: 
Loss:  1.966920018196106
Accuracy:  30.62 %
Add 1 parameter: 
Loss:  0.65035080909729
Accuracy:  79.31 %
Add 2 parameter: 
Loss:  0.3533753454685211
Accuracy:  88.64 %
Add 3 parameter: 
Loss:  0.36316439509391785
Accuracy:  88.16 %
Add 4 parameter: 
Loss:  0.3226277530193329
Accuracy:  89.83 %
Add 5 parameter: 
Loss:  0.31805434823036194
Accuracy:  90.91 %


# **Compare Time Cost to Train**

In [74]:
print("Time cost for each epoch (total 50 epochs) for model with 0 additional parameter: ", np.round(duration_0/50, 2), "seconds")
print("Time cost for each epoch (total 50 epochs) model with 1 additional parameter: ", np.round(duration_1/50, 2), "seconds")
print("Time cost for each epoch (total 50 epochs) model with 2 additional parameter: ", np.round(duration_2/50, 2), "seconds")
print("Time cost for each epoch (total 50 epochs) model with 3 additional parameter: ", np.round(duration_3/50, 2), "seconds")
print("Time cost for each epoch (total 50 epochs) model with 4 additional parameter: ", np.round(duration_4/50, 2), "seconds")
print("Time cost for each epoch (total 50 epochs) model with 5 additional parameter: ", np.round(duration_5/50, 2), "seconds")

Time cost for each epoch (total 50 epochs) for model with 0 additional parameter:  0.72 seconds
Time cost for each epoch (total 50 epochs) model with 1 additional parameter:  0.74 seconds
Time cost for each epoch (total 50 epochs) model with 2 additional parameter:  0.68 seconds
Time cost for each epoch (total 50 epochs) model with 3 additional parameter:  0.71 seconds
Time cost for each epoch (total 50 epochs) model with 4 additional parameter:  0.8 seconds
Time cost for each epoch (total 50 epochs) model with 5 additional parameter:  0.7 seconds
